In [11]:
from __future__ import print_function
import pandas as pd
import numpy as np
from pyspark import SparkContext
from pyspark.sql import SparkSession
# $example on$
from pyspark.mllib.feature import HashingTF, IDF

In [2]:
 sc = SparkContext(appName="TFIDFExample")  # SparkContext

    # $example on$
    # Load documents (one per line).
documents = sc.textFile("farm-ads.txt").map(lambda line: line.split(" "))

In [3]:
hashingTF = HashingTF()
tf = hashingTF.transform(documents)

# While applying HashingTF only needs a single pass to the data, applying IDF needs two passes:
# First to compute the IDF vector and second to scale the term frequencies by IDF.
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

In [23]:
tfidf.collect()

[SparseVector(1048576, {84584: 6.2569, 140248: 15.2865, 325835: 13.9002, 405860: 7.6432, 407524: 1.253, 412328: 6.5446, 466741: 5.9385, 589703: 1.6162, 832052: 7.2378, 877522: 0.6352}),
 SparseVector(1048576, {9964: 4.6728, 67315: 3.7413, 190721: 5.392, 243172: 5.1175, 259627: 2.7757, 380043: 4.2093, 423293: 2.9941, 469874: 2.8726, 474923: 1.9563, 498232: 3.5742, 544926: 4.0057, 591364: 4.781, 599720: 4.5752, 649837: 4.0189, 658475: 0.769, 671973: 4.3661, 742405: 2.7567, 757764: 3.2612, 850711: 5.6973, 890175: 3.4844, 984811: 4.4862}),
 SparseVector(1048576, {99761: 7.6432, 189117: 2.8983, 222063: 7.6432, 261616: 7.6432, 278555: 3.0232, 348859: 7.2378, 352790: 7.6432, 528275: 10.892, 528360: 3.3735, 560704: 3.7114, 630681: 1.6694, 658475: 0.769, 685200: 1.2631, 741810: 3.346, 777769: 5.6283, 855610: 7.6432, 874347: 13.4539, 895490: 3.5742, 938726: 6.3905, 958529: 7.6432, 961863: 7.6432, 1011763: 3.2184, 1033510: 7.2378, 1034640: 7.6432}),
 SparseVector(1048576, {16419: 4.8399, 26230: 5

In [4]:
idfIgnore = IDF(minDocFreq=2).fit(tf)
tfidfIgnore = idfIgnore.transform(tf)

In [5]:
a= tfidf.collect()

In [14]:
df = tfidf.map(lambda v: (v, )).toDF(["features"])

In [18]:
training, test = df.randomSplit([0.6, 0.4], seed=12345)


In [19]:
training.show()

+--------------------+
|            features|
+--------------------+
|(1048576,[0,229,5...|
|(1048576,[0,229,1...|
|(1048576,[0,944,1...|
|(1048576,[0,944,3...|
|(1048576,[0,2980,...|
|(1048576,[0,8673,...|
|(1048576,[0,9672,...|
|(1048576,[0,9964,...|
|(1048576,[0,9964,...|
|(1048576,[0,11612...|
|(1048576,[0,14079...|
|(1048576,[0,14079...|
|(1048576,[0,14079...|
|(1048576,[0,14079...|
|(1048576,[0,14079...|
|(1048576,[0,14763...|
|(1048576,[0,14763...|
|(1048576,[0,14763...|
|(1048576,[0,15553...|
|(1048576,[0,15553...|
+--------------------+
only showing top 20 rows



In [21]:
from pyspark.ml.classification import LogisticRegression
new_schema = ArrayType(DoubleType(), containsNull=False)

df_foo = udf(lambda x:x, new_schema)
training = training.withColumn("topic_vector_fix_dim",udf_foo("topic_vector_fix_dim"))

log_reg = LogisticRegression(featuresCol='features', labelCol='label')
logr_model = log_reg.fit(training)

NameError: name 'ArrayType' is not defined